# 0. Prepreocessing & installation


## 0-1. Mount your google drive to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 0-2. Install kaldi main

In [ ]:
!pip install kora -q
import kora.install.kaldi
from kora.tree import path_tree; path_tree('/opt/kaldi')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 739.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00


## 0-3. Install pip dependency

In [ ]:
!pip install pypinyin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.9 MB/s eta 0:00:00


## 0-4. Install kaldi dependency & set path

In [ ]:
import os
import subprocess
import pandas
os.chdir('/opt/kaldi/tools/')
# point to the dir you mount
#/content/drive/MyDrive/BIIC_ASR
biic_asr_main_dir = '/content/drive/MyDrive/Lab2/BIIC_ASR/'

In [ ]:
!extras/check_dependencies.sh

extras/check_dependencies.sh: WARNING python 2.7 is not the default python. We fixed this by adding a correct symlink more prominently on the path.
 ... If you really want to use python 3.10.12 as default, add an empty file /opt/kaldi/tools/python/.use_default_python and run this script again.
extras/check_dependencies.sh: Intel MKL is not installed. Run extras/install_mkl.sh to install it.
 ... You can also use other matrix algebra libraries. For information, see:
 ...   http://kaldi-asr.org/doc/matrixwrap.html


In [ ]:
!apt-get install sox libtool subversion python2.7 dos2unix gawk
!extras/install_irstlm.sh
!extras/install_kaldi_lm.sh
!cp /content/drive/Shareddrives/11220-EE6640-SSPlab/BIIC_ASR/fix_tool/install_srilm.sh extras/install_srilm.sh
!cp /content/drive/Shareddrives/11220-EE6640-SSPlab/BIIC_ASR/fix_tool/install_liblbfgs.sh extras/install_liblbfgs.sh
!cp /content/drive/Shareddrives/11220-EE6640-SSPlab/BIIC_ASR/fix_tool/srilm.tgz srilm.tgz
!extras/install_srilm.sh 11220-EE6640-SSPlab 11220-EE6640-SSPlab EE6640@nthu.edu.tw
!bash env.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtool is already the newest version (2.4.6-15build2).
dos2unix is already the newest version (7.4.2-2).
gawk is already the newest version (1:5.1.0-1ubuntu0.1).
python2.7 is already the newest version (2.7.18-13ubuntu1.2).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
subversion is already the newest version (1.14.1-3ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
****() Installing IRSTLM
****() Assuming IRSTLM is already installed. Please delete
****() the directory ./irstlm if you need us to download
****() the sources again.
Installing kaldi_lm
make: Nothing to be done for 'all'.
Installation of kaldi_lm finished successfully
Please source tools/env.sh in your path.sh to enable it
cp: cannot stat '/content/drive/Shareddrives/11220-EE6640-SSPlab/BIIC_ASR/fix_tool/install_srilm.sh': No such file or directory
cp: cannot 

## 0-5. cd to workspace

In [ ]:
os.chdir(biic_asr_main_dir)
!chmod -R 777 ./*

## 0-6. Choose dataset
Digitss: Only contained 1-10 in chinese \\


In [ ]:
# digits config
!cp kaldi_config_digits.txt kaldi_config.txt
# phone numbers config
#!cp kaldi_config_digits_phone.txt kaldi_config.txt
#!cp kaldi_config_phone_number_decode.txt kaldi_config.txt

# 1. Training processing

In [ ]:
!bash Stage1_prepare.sh

In [ ]:
!bash Stage2_LM_training.sh

In [ ]:
!bash Stage3_mfcc_extract.sh

In [ ]:
!bash Stage4_mono_train.sh

In [ ]:
!bash Stage5_tri1.sh


In [ ]:
!bash Stage6_tri2.sh

In [ ]:
!bash Stage7_tri3.sh

In [ ]:
!bash Stage8_tri4.sh

In [ ]:
!bash Stage9_tri5.sh

In [ ]:
!bash Stage10_decode_result.sh

# 2. Decoding processing

In [ ]:
!rm kaldi_config.txt
#kaldi_config_school_number_decode.txt
!cp kaldi_config_phone_number_decode.txt kaldi_config.txt
#!cp kaldi_config_school_number_decode.txt kaldi_config.txt

In [ ]:
!python make_prepare_data_case_0.py
!bash local/prepare_data_decode.sh
!echo "Finish prepare data"

Preparing decoding data
Finish prepare data


In [ ]:
!bash mfcc_decode_extract.sh
!echo "Finish feature extraction"

mfcc_decode_extract.sh: making mfccs
steps/make_mfcc_pitch.sh --cmd run.pl --mem 32G --nj 1 data/decode exp/make_mfcc/decode mfcc
utils/validate_data_dir.sh: Successfully validated data-directory data/decode
steps/make_mfcc_pitch.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
Succeeded creating MFCC & Pitch features for decode
steps/compute_cmvn_stats.sh data/decode exp/make_mfcc/decode mfcc
Succeeded creating CMVN stats for decode
fix_data_dir.sh: kept all 31 utterances.
fix_data_dir.sh: old files are kept in data/decode/.backup
Finish feature extraction


In [ ]:
!python make_tri5_run_decode.py
!bash run_decode.sh
!echo "Finish decoding"

steps/decode_fmllr.sh --nj 1 --cmd run.pl --mem 32G --skip-scoring false exp/my_mono2/graph data/decode exp/my_mono2/phone_numbers_decode_result/
steps/decode.sh --scoring-opts  --num-threads 1 --skip-scoring false --acwt 0.083333 --nj 1 --cmd run.pl --mem 32G --beam 10.0 --model exp/my_mono2/final.mdl --max-active 2000 exp/my_mono2/graph data/decode exp/my_mono2/phone_numbers_decode_result.si
decode.sh: feature type is delta
steps/diagnostic/analyze_lats.sh --cmd run.pl --mem 32G exp/my_mono2/graph exp/my_mono2/phone_numbers_decode_result.si
steps/diagnostic/analyze_lats.sh: see stats in exp/my_mono2/phone_numbers_decode_result.si/log/analyze_alignments.log
Overall, lattice depth (10,50,90-percentile)=(1,4,23) and mean=20.3
steps/diagnostic/analyze_lats.sh: see stats in exp/my_mono2/phone_numbers_decode_result.si/log/analyze_lattice_depth_stats.log
+ steps/score_kaldi.sh --cmd 'run.pl --mem 32G' data/decode exp/my_mono2/graph exp/my_mono2/phone_numbers_decode_result.si
steps/score_kal

In [ ]:
!python decode2txt_tri5.py

Decode results are in paths:phone_numbers_deocde
%WER 22.02 [ 61 / 277, 4 ins, 37 del, 20 sub ] exp/my_mono2/phone_numbers_decode_result/cer_16_0.0
%WER 22.02 [ 61 / 277, 4 ins, 37 del, 20 sub ] exp/my_mono2/phone_numbers_decode_result/wer_16_0.0


# 3. Save model for next time

In [ ]:
!cp -r exp/mono exp/my_mono2 # save acoustic model
!cp -r data/lang data/lang_part_of_gamania # save language model
!cp -r my_lexicon.txt lexcion_part_of_gamania.txt # save language model